imports

In [50]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import models, transforms
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

In [51]:
data_path = 'filtered_merged_elephantidae.csv'
images_path = 'data/images'

df = pd.read_csv(data_path)

### Préparer les colonnes nécessaires pour la classification

In [52]:
# Exemple : Classification basée sur le taxon_species_name_x
df = df[['id', 'taxon_species_name_x']].dropna()
df['taxon_species_name_x'].value_counts()


taxon_species_name_x
Loxodonta africana    1791
Elephas maximus        142
Loxodonta cyclotis      44
Name: count, dtype: int64

### Encoder les classes

In [53]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['taxon_species_name_x'])

### Augmentation des minorités

In [54]:
minority_classes = ['Elephas maximus', 'Loxodonta cyclotis']

# Augmentation de données pour les classes minoritaires
minority_classes = ['Elephas maximus', 'Loxodonta cyclotis']

def augment_image(image):
    augmentations = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0))
    ])
    return augmentations(image)

for index, row in df.iterrows():
    if row['taxon_species_name_x'] in minority_classes:
        img_path = os.path.join(images_path, f"{row['id']}.jpg")
        if os.path.exists(img_path):
            image = Image.open(img_path).convert("RGB")
            for i in range(5):  # Créez 5 augmentations par image
                augmented_image = augment_image(image)
                augmented_image.save(os.path.join(images_path, f"{row['id']}_aug_{i}.jpg"))

# Mise à jour du dataframe pour inclure les images augmentées
augmented_data = []
for index, row in df.iterrows():
    if row['taxon_species_name_x'] in minority_classes:
        for i in range(5):
            new_id = f"{row['id']}_aug_{i}"
            augmented_data.append({'id': new_id, 'taxon_species_name_x': row['taxon_species_name_x'], 'label': row['label']})
df = pd.concat([df, pd.DataFrame(augmented_data)], ignore_index=True)

# Répartition équilibrée (sur-échantillonnage des minorités)
class_counts = df['taxon_species_name_x'].value_counts()
max_class_count = class_counts.max()

balanced_df = pd.concat([
    df[df['taxon_species_name_x'] == cls].sample(n=max_class_count, replace=True, random_state=42)
    for cls in class_counts.index
])

### Split des données en train/val/test


In [55]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

### Dataset pour charger les images

In [56]:
class ElephantDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_path = os.path.join(self.image_dir, f"{row['id']}.jpg")
        image = Image.open(img_path).convert("RGB")
        label = row['label']

        if self.transform:
            image = self.transform(image)

        return image, label

### Transformation pour ResNet

In [57]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

### Dataloaders

In [58]:
train_dataset = ElephantDataset(train_df, images_path, transform=transform)
val_dataset = ElephantDataset(val_df, images_path, transform=transform)
test_dataset = ElephantDataset(test_df, images_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Charger le modèle ResNet pré-entraîné



In [59]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(label_encoder.classes_))

c:\Users\josep\Documents\GitHub\MLforDE_project\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\josep\Documents\GitHub\MLforDE_project\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Envoyer le modèle sur le GPU si disponible


In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

### Configurer la loss avec pondération pour les classes


In [61]:
class_counts = balanced_df['taxon_species_name_x'].value_counts()
weights = [1.0 / class_counts[label_encoder.classes_[i]] for i in range(len(label_encoder.classes_))]
weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

### Configurer la loss et l'optimiseur

In [62]:

optimizer = optim.Adam(model.parameters(), lr=0.001)


### Fonction d'entraînement


In [63]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Accuracy: {100 * correct / total:.2f}%")

### Entraîner le modèle

In [ ]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=2)


Epoch 1/5, Train Loss: 0.5819, Val Loss: 0.8645, Val Accuracy: 65.64%
Epoch 2/5, Train Loss: 0.4391, Val Loss: 0.4666, Val Accuracy: 86.94%
Epoch 3/5, Train Loss: 0.3958, Val Loss: 0.5331, Val Accuracy: 79.38%


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


In [ ]:
evaluate_model(model, test_loader)

In [ ]:
# Chemin pour sauvegarder le modèle (poids uniquement)
model_save_path = "classification_elephants.pth"

# Sauvegarder uniquement les poids du modèle
torch.save(model.state_dict(), model_save_path)
print(f"Modèle sauvegardé à : {model_save_path}")